In [32]:
from openai import OpenAI

YOUR_API_KEY = "your openai api key here"
client = OpenAI(api_key = YOUR_API_KEY)

In [2]:
import html
import re

chi_pattern = re.compile(r'【[^】]*】')
num_pattern = re.compile(r'\d+\)\s+')

def preprocess(text):
    text = html.unescape(text)

    # text = re.sub(chi_pattern1, "", text)
    text = re.sub(chi_pattern, "", text)
    text = re.sub(num_pattern, "", text)

    text = re.sub(r'\s+', ' ', text)

    return text

In [3]:
example_docs = {1: [], 2: [], 3: [], 4: []}

with open('few_shot_data/few_shot_example_docs.txt', 'r', encoding='utf-8') as f:
    for line in f:
        doc_name, label = line.strip().split(', ')
        label = int(label)
        example_docs[label].append(doc_name)

example_docs

{1: ['kua_14201003_001',
  'kda_11801016_004',
  'kqa_10805002_002',
  'kma_11903015_001',
  'kka_13401027_001'],
 2: ['ksa_11604016_001',
  'kda_11103006_007',
  'kpa_11602014_001',
  'kna_12603017_004',
  'kda_10411014_006'],
 3: ['kua_11809002_002',
  'kka_11210015_002',
  'kva_10206115_001',
  'krb_11403008_003',
  'kka_10406014_002'],
 4: ['kma_11104010_004',
  'kda_11710013_001',
  'kua_11701029_001',
  'kka_12705027_001',
  'kda_12510013_002']}

In [4]:
test_docs_with_ans = []

with open('few_shot_data/test_set_docs.txt', 'r', encoding='utf-8') as f:
    for line in f:
        test_docs_with_ans.append(line.strip().split(', '))

test_docs = [doc for doc, _ in test_docs_with_ans]
print(test_docs)
print(len(test_docs))

['kca_10105027_001', 'kca_10108022_001', 'kca_10211015_002', 'kca_10305021_001', 'kca_10407020_001', 'kca_10412030_002', 'kca_10801020_001', 'kca_10809018_002', 'kca_10904101_001', 'kca_11006025_001', 'kca_11010026_002', 'kca_11204015_001', 'kca_11410008_002', 'kca_11806022_003', 'kda_10101003_002', 'kda_10111017_002', 'kda_10201128_006', 'kda_10204004_006', 'kda_10205003_001', 'kda_10303014_001', 'kda_10310004_003', 'kda_10506029_001', 'kda_10508022_001', 'kda_10510016_004', 'kda_10707028_002', 'kda_10806024_002', 'kda_10809029_002', 'kda_10904001_001', 'kda_10905002_001', 'kda_11003006_001', 'kda_11011009_002', 'kda_11104025_003', 'kda_11201011_002', 'kda_11204004_002', 'kda_11209017_002', 'kda_11311010_002', 'kda_11401026_001', 'kda_11403008_001', 'kda_11510008_002', 'kda_11607013_004', 'kda_11701015_002', 'kda_11703004_005', 'kda_11707014_002', 'kda_11711013_002', 'kda_11902005_001', 'kda_11904026_001', 'kda_12012018_004', 'kda_12102124_001', 'kda_12206021_003', 'kda_12306009_001',

In [8]:
import random

random.seed(42)

two_shot_ex = {}
for ground_truth, docs in example_docs.items():
    for doc in random.sample(docs, 2):
        two_shot_ex[doc] = ground_truth

pre = {'kua_14201003_001': 1,
 'kka_13401027_001': 1,
 'kpa_11602014_001': 2,
 'kda_11103006_007': 2,
 'kka_11210015_002': 3,
 'kka_10406014_002': 3,
 'kma_11104010_004': 4,
 'kda_12510013_002': 4}

print(pre == two_shot_ex) # check if test data is same as pre-defined data
two_shot_ex

True


{'kua_14201003_001': 1,
 'kka_13401027_001': 1,
 'kpa_11602014_001': 2,
 'kda_11103006_007': 2,
 'kka_11210015_002': 3,
 'kka_10406014_002': 3,
 'kma_11104010_004': 4,
 'kda_12510013_002': 4}

In [9]:
import os
import json
import html
import codecs

root_path = "data/pp_kor_docs_json/"

example_text = {}
for root, dirs, files in os.walk(root_path):
    for one_file in sorted(files):
        if ".json" in one_file:
            with codecs.open(os.path.join(root, one_file), "r", "utf-8") as kor_json_f:
                kor_files_content = json.load(kor_json_f)

                for each_kor_doc_name, each_kor_doc_content in sorted(kor_files_content.items(), key=lambda x: x[0]):
                    if each_kor_doc_name not in two_shot_ex:
                        continue
                    idx = two_shot_ex[each_kor_doc_name]

                    kor_title = html.unescape(each_kor_doc_content[0])

                    kor_body = kor_title + " " + " ".join(each_kor_doc_content[1])
                    kor_body = preprocess(kor_body)

                    try:
                        example_text[idx].append(kor_body)
                    except:
                        example_text[idx] = [kor_body]

example_text # example texts using for few-shot learning

{2: ['한성시·회시에 삼군 오원을 임명하여 보내달라고 예조에서 건의하다 예조에서 계하기를, “문과(文科)의 관시(館試)에 양현고 녹사(養賢庫錄事)로 하여금 협책(挾冊) 여부를 수금(搜禁)하게 하는 것은 온당하지 않사오니, 청하건대 한성시(漢城試)·회시(會試)의 예(例)에 의하여 삼군 오원(五員)을 임명하여 보내도록 하소서.” 하니, 그대로 따랐다.',
  '비국이 전라 감사 원두표의 유임을 청하니 윤허하다 비국이 아뢰기를, “전라 감사 원두표(元斗杓)는 임기가 만료되어 체차해야 하는데, 이 직임을 잘 수행할 만한 적임자를 얻기가 어려우니, 1년을 더 유임시키소서.” 하니, 따랐다.'],
 4: ['통신사 변효문이 돌아와 일본에서의 일을 치계하다 일본국(日本國)에 갔던 통신사 변효문(卞孝文)이 돌아와 경상도 옥포(玉浦)에 이르러 치계(馳啓)하기를, “신 등이 처음에 대마도(對馬島)에 이르니, 종정성(宗貞盛)이 하사한 물건을 받고 사배(四拜)를 행하여 사의를 표시하였고, 인하여 사람을 보내어 향도(鄕道)를 삼아 적간관(赤間關)의 대내(大內) 교홍(敎弘)이 있는 데로 출발시켰는데, 곧 적기포(赤崎浦)에 이르니 영후(迎候)하였습니다. 이튿날 사물(賜物)을 받는데, 사물(賜物)이 장차 이르매, 교홍(敎弘)이 뜰 아래에 서서 국궁하여 맞고, 당에 올라 꿇어앉아서 받되 고두(扣頭)하였으며, 인하여 신 등에게 의복을 주고 후하게 식물(食物)을 주었습니다. 신이 교홍에게 이르기를, ‘우리 전하께서 고 대내전(大內殿)의 성의를 추념(追念)하시어 우리들에게 명하여 전(奠)을 드리게 하였습니다.’ 하니, 대답하기를, ‘중첩하여 후한 은혜를 받으니 감사한 것을 다하기 어렵습니다. 다만 사신이 아직 국왕께 경례(慶禮)를 행하지도 않았는데, 먼저 전례(奠禮)를 받으면 국왕이 나쁘게 여길까 두려우니, 청하건대, 회환할 때에 치전(致奠)하소서.’ 하였습니다. 그리고, 교홍이 신 등의 온 뜻을 경도(京都)에 치보(馳報)하고, 배 4척을 발하여 호송하게 하여 미로(尾路)에 이르니, 대관(代

In [26]:
except_docs = []
with open('few_shot_data/except_docs.txt', 'r', encoding='utf-8') as f:
    for line in f:
        except_docs.append(line.strip())

print(except_docs) # this documents are not used for test because of vague date

['kja_105110??_001', 'kja_105110??_002', 'kja_105110??_003', 'kna_10212099_001', 'kna_108050??_001', 'kna_108050??_002', 'kna_108050??_003', 'kna_108050??_004', 'kna_108050??_005', 'kna_108050??_006', 'kna_108070??_001', 'kna_108070??_002', 'kna_109110??_001', 'kna_114070??_001', 'kna_114070??_002', 'kna_114070??_003', 'kna_114080??_001', 'kna_116050??_001', 'kna_117090??_001', 'kna_118010??_001', 'kna_118010??_002', 'kna_118010??_003', 'kna_118020??_001', 'kna_118020??_002', 'kna_118030??_001', 'kna_118120??_001', 'kna_119020??_001', 'kna_119090??_001', 'kna_130120??_001', 'kna_130120??_002', 'kna_130120??_003', 'kna_130120??_004', 'kna_130120??_005', 'kna_131070??_001', 'kna_131070??_002', 'kva_102030??_001']


In [30]:
import pandas as pd
import csv

file_name = "kings_decisions_gpt_4o_two_shot.csv"
if os.path.exists(file_name):
    print(f"Warning: The file '{file_name}' already exists. The file was not saved.")
else:
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        writer.writerow(["doc_name", "gpt_answer"])

In [34]:
def append_row_to_csv(doc_name, gpt_answer):
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([doc_name, gpt_answer])

In [44]:
import pandas as pd

for root, dirs, files in os.walk(root_path):
    for one_file in sorted(files):
        if ".json" in one_file:
            print("Do {}".format(one_file))

            with codecs.open(os.path.join(root, one_file), "r", "utf-8") as kor_json_f:
                kor_files_content = json.load(kor_json_f)

                for each_kor_doc_name, each_kor_doc_content in sorted(kor_files_content.items(), key=lambda x: x[0]):
                    if each_kor_doc_name in test_docs or each_kor_doc_name in except_docs or each_kor_doc_name in two_shot_ex:
                        append_row_to_csv(each_kor_doc_name, '-')
                        continue

                    kor_title = html.unescape(each_kor_doc_content[0])

                    kor_body = kor_title + " " + " ".join(each_kor_doc_content[1])
                    kor_body = preprocess(kor_body)
                    
                    response = client.chat.completions.create(
                        model="gpt-4o-2024-08-06",
                        messages=[
                            {"role": "system", "content": "As a historian, you need to read the Annals of the Joseon Dynasty translated into modern Korean and determine the king's decision included in the document. Classify it into four types:  three for the presence of different types of decisions and one for the absence of a decision."},
                            {"role": "user", "content": "When a document is given, classify it into one of the following four types. 1. Direct Order, 2. Agreement, 3. Disagreement, 4. Missing"},
                            {"role": "user", "content": "The four types are as follows; 1. Direct Order: The king issues a command on his own without discussing with court officials, 2. Agreement: The king discusses with court officials and follows their suggestions, 3. Disagreement: The king discusses with court officials and rejects their suggestions, 4. Missing: The king's decision is not present."},
                            {"role": "user", "content": f"Here are examples for each type.\n1. Direct Order: {example_text[1][0]}\n2. Agreement: {example_text[2][0]}\n3. Disagreement: {example_text[3][0]}\n4. Missing: {example_text[4][0]}\n1. Direct Order: {example_text[1][1]}\n2. Agreement: {example_text[2][1]}\n3. Disagreement: {example_text[3][1]}\n4. Missing: {example_text[4][1]}"},
                            {"role": "user", "content": f"Here is the document you need to read and classify. \nDocument: {kor_body}"},
                            {"role": "user", "content": "You must answer with one of '1. Direct Order', '2. Agreement', '3. Disagreement', '4. Missing'. If it is difficult to judge, choose '4. Missing'"},
                            {"role": "user", "content": "Even if you are not sure, do not add any other words or symbols, only answer with '1. Direct Order', '2. Agreement', '3. Disagreement', or '4. Missing'"}
                        ]
                    )

                    gpt_answer = response.choices[0].message.content.strip()
                    append_row_to_csv(each_kor_doc_name, gpt_answer)

Do kaa_101.json
Do kaa_102.json
Do kaa_103.json
Do kaa_104.json
Do kaa_105.json
Do kaa_106.json
Do kaa_107.json
Do kba_101.json
Do kba_102.json
Do kca_101.json
Do kca_102.json
Do kca_103.json
Do kca_104.json
Do kca_105.json
Do kca_106.json
Do kca_107.json
Do kca_108.json
Do kca_109.json
Do kca_110.json
Do kca_111.json
Do kca_112.json
Do kca_113.json
Do kca_114.json
Do kca_115.json
Do kca_116.json
Do kca_117.json
Do kca_118.json
Do kda_100.json
Do kda_101.json
Do kda_102.json
Do kda_103.json
Do kda_104.json
Do kda_105.json
Do kda_106.json
Do kda_107.json
Do kda_108.json
Do kda_109.json
Do kda_110.json
Do kda_111.json
Do kda_112.json
Do kda_113.json
Do kda_114.json
Do kda_115.json
Do kda_116.json
Do kda_117.json
Do kda_118.json
Do kda_119.json
Do kda_120.json
Do kda_121.json
Do kda_122.json
Do kda_123.json
Do kda_124.json
Do kda_125.json
Do kda_126.json
Do kda_127.json
Do kda_128.json
Do kda_129.json
Do kda_130.json
Do kda_131.json
Do kda_132.json
Do kea_100.json
Do kea_101.json
Do kea_1

KeyboardInterrupt: 

In [ ]:
ans_dict = {'1. Direct Order': 1, '2. Agreement': 2, '3. Disagreement': 3, '4. Missing': 4}